# Generate AES key with Argon2id KDF using salt (entropy) from Trezor

In [78]:
password = "Super secure password!!11"

## Connecting to Trezor device

We will use the Trezor hardware wallet to get random bytes of entropy when needed (Trezor will prompt you each time):

In [79]:
from trezorlib import misc, ui
from trezorlib.client import TrezorClient
from trezorlib.transport import get_transport

try:
    client = TrezorClient(get_transport(), ui=ui.ClickUI())
except Exception as e:
    print(e)
    exit()

Test if it works:

In [80]:
misc.get_entropy(client, 16)

Please confirm action on your Trezor device.


b'\x85\x8072\xb7\xd8\xcb\x99\x88d\xca\xf5\xf6V\xcf|'

## Argon2 KDF

Use Argon2id to generate a 32 byte password hash that we will use for AES:

In [81]:
import argon2

# store salt alongside ciphertext
salt = misc.get_entropy(client, 16)

password_hash = argon2.low_level.hash_secret_raw(
    str.encode(password), salt,
    time_cost=16,
    memory_cost=2**16,
    parallelism=2,
    hash_len=32, # hash_len must be 32 for AES
    type=argon2.low_level.Type.ID
)

## AES 

We can now initialize AES with our encryption key we generated with Argon2id before + using the entropy from Trezor for the AES initialization vector:

In [82]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = password_hash
iv = misc.get_entropy(client, 16)

cipher = Cipher(algorithms.AES(key), modes.CBC(iv))

### Encryption

Use the key to encrypt some data:

In [83]:
encryptor = cipher.encryptor()

enc = encryptor.update(b"a secret message") + encryptor.finalize()

print(enc)

b'x\xa7\x86\xa0\xe9F\x0bwL\xd4\x80L\x1f\xa7eD'


### Decryption

Use the key to decrypt the data:

In [84]:
decryptor = cipher.decryptor()

dec = decryptor.update(enc) + decryptor.finalize()

print(dec)

b'a secret message'
